PASOS:

1. Tener en cuenta este enlace: https://developers.google.com/drive/activity/v2/quickstart/python?hl=es-419
2. Habilitar la API
3. Usar Credencial 4. Descargar el archivo JSON
4. Navegar en el Anaconda Prompt hasta la dirección de app2
5. Ejecutar el archivo quickstart así: python quickstart.py
6. Correr el loop. CLIENT_SECRET_FILE = token.json

In [ ]:
from Google import Create_Service
import pandas as pd
from google.oauth2.credentials import Credentials
from googleapiclient.errors import HttpError
from googleapiclient.discovery import build
from googleapiclient import errors


In [44]:
CLIENT_SECRET_FILE = 'credentials.json'
API_NAME = 'drive'
API_VERSION = 'v3'
SCOPES = ['https://www.googleapis.com/auth/drive']

service = Create_Service(CLIENT_SECRET_FILE, API_NAME, API_VERSION, SCOPES)

# Update Sharing Setting
file_id = '1FobI3q1teoi5nRucHOjrvnSiPq8Lp7MC'

request_body = {
    'role': 'reader',
    'type': 'anyone'
}

response_permission = service.permissions().create(
    fileId=file_id,
    body=request_body
).execute()

print(response_permission)


# Print Sharing URL
response_share_link = service.files().get(
    fileId=file_id,
    fields='webViewLink'
).execute()

print(response_share_link)

credentials.json-drive-v3-(['https://www.googleapis.com/auth/drive'],)
['https://www.googleapis.com/auth/drive']
drive service created successfully
{'kind': 'drive#permission', 'id': 'anyoneWithLink', 'type': 'anyone', 'role': 'reader', 'allowFileDiscovery': False}
{'webViewLink': 'https://drive.google.com/file/d/1FobI3q1teoi5nRucHOjrvnSiPq8Lp7MC/view?usp=drivesdk'}


In [50]:
# Define the ID of the main folder containing the subfolders with PDF files
main_folder_id = '16AcV9UcAAuJVaee-WmQWh9O8Ul9zWP3o'

# Define the Google Drive API parameters
CLIENT_SECRET_FILE = 'token.json'
API_NAME = 'drive'
API_VERSION = 'v3'
SCOPES = ['https://www.googleapis.com/auth/drive']

# Define the service object for the Google Drive API
creds = None
creds = Credentials.from_authorized_user_file(CLIENT_SECRET_FILE, SCOPES)
service = build(API_NAME, API_VERSION, credentials=creds)

# Define the query to retrieve all subfolders within the main folder
query = f"'{main_folder_id}' in parents and mimeType='application/vnd.google-apps.folder' and trashed = false"
subfolders = []

try:
    # Execute the query and retrieve all subfolders within the main folder
    page_token = None
    while True:
        response = service.files().list(q=query, fields='nextPageToken, files(id, name)', pageToken=page_token).execute()
        subfolders += response.get('files', [])
        page_token = response.get('nextPageToken', None)
        if page_token is None:
            break
except errors.HttpError as error:
    print(f'An error occurred: {error}')
    subfolders = None

# Create an empty list to store the file dictionaries
file_list = []

if subfolders is not None:
    # Loop through each subfolder and retrieve PDF files within it
    for subfolder1 in subfolders:
        subfolder1_id = subfolder1['id']
        query = "mimeType='application/vnd.google-apps.folder' and trashed = false and parents in '" + subfolder1_id + "'"
        subfolders2 = []

        try:
            # Execute the query and retrieve all subfolders within the first subfolder
            page_token = None
            while True:
                response = service.files().list(q=query, fields='nextPageToken, files(id, name)', pageToken=page_token).execute()
                subfolders2 += response.get('files', [])
                page_token = response.get('nextPageToken', None)
                if page_token is None:
                    break
        except errors.HttpError as error:
            print(f'An error occurred: {error}')
            subfolders2 = None

        if subfolders2 is not None:
            # Loop through each subfolder and retrieve PDF files within it
            for subfolder2 in subfolders2:
                subfolder2_id = subfolder2['id']
                query = "mimeType='application/pdf' and trashed = false and parents in '" + subfolder2_id + "'"

                try:
                    # Execute the query and retrieve all PDF files in the second subfolder
                    results = service.files().list(q=query, fields="nextPageToken, files(id, name, webViewLink)").execute()
                    items = results.get("files", [])

                    # Add the file dictionaries to the file list
                    file_list += [{'Folder1': subfolder1['name'], 'Folder2': subfolder2['name'], 'Name': item['name'], 'URL': item['webViewLink']} for item in items]

                except HttpError as error:
                    print(f'An error occurred: {error}')
                    items = None

# Create a DataFrame from the file list
df = pd.DataFrame(file_list)
print(df)

# Export the DataFrame to an Excel file
file_name = 'pdf_files.xlsx'
df.to_excel(file_name, index=False)
print(f'DataFrame saved to {file_name}.')


        Folder1                Folder2                               Name  \
0       UCAYALI  5802-2019-CG_GRUC-SCE  5802-2019-CG_GRUC-SCE-informe.pdf   
1       UCAYALI  5802-2019-CG_GRUC-SCE  5802-2019-CG_GRUC-SCE-resumen.pdf   
2       UCAYALI       296-2018-CG_GRUC       296-2018-CG_GRUC-informe.pdf   
3       UCAYALI       296-2018-CG_GRUC       296-2018-CG_GRUC-resumen.pdf   
4       UCAYALI   2036-2019-CG_GRUC-AC   2036-2019-CG_GRUC-AC-informe.pdf   
...         ...                    ...                                ...   
1695  CAJAMARCA        015-2019-2-0372        015-2019-2-0372-informe.pdf   
1696  CAJAMARCA        014-2020-2-0374        014-2020-2-0374-resumen.pdf   
1697  CAJAMARCA        014-2020-2-0374        014-2020-2-0374-informe.pdf   
1698  CAJAMARCA        003-2017-2-0378        003-2017-2-0378-resumen.pdf   
1699  CAJAMARCA        003-2017-2-0378        003-2017-2-0378-informe.pdf   

                                                    URL  
0     https://dri

In [51]:
data = pd.read_excel( 'pdf_files.xlsx' )

In [54]:
data = data[ [ 'Name', 'URL' ] ]
data

,Name,URL
0,5802-2019-CG_GRUC-SCE-informe.pdf,https://drive.google.com/file/d/1HOOqoV9bE0Gbi...
1,5802-2019-CG_GRUC-SCE-resumen.pdf,https://drive.google.com/file/d/1HFOOlRdN9Uvbj...
2,296-2018-CG_GRUC-informe.pdf,https://drive.google.com/file/d/1HCyaEyLnbsCmb...
3,296-2018-CG_GRUC-resumen.pdf,https://drive.google.com/file/d/1HCcybYJxpf5lM...
4,2036-2019-CG_GRUC-AC-informe.pdf,https://drive.google.com/file/d/1HBLbnxPC_byhN...
...,...,...
1695,015-2019-2-0372-informe.pdf,https://drive.google.com/file/d/1l9Vd17vI13u6i...
1696,014-2020-2-0374-resumen.pdf,https://drive.google.com/file/d/1kvTwhDpoA4ZNt...
1697,014-2020-2-0374-informe.pdf,https://drive.google.com/file/d/1kvR6X3q7prVRr...
1698,003-2017-2-0378-resumen.pdf,https://drive.google.com/file/d/1l5DmH_GWLkNrE...


In [68]:
# create a new dataframe with rows that end with "-informe.pdf"
data_informe = data[ data[ 'Name' ].str.endswith( '-informe.pdf' ) ]
data_informe[ 'Name' ] = data_informe['Name'].str.replace( '-informe.pdf', '' )
data_informe = data_informe.rename( columns = { 'URL': 'Informe_Completo' } )

# create a new dataframe with rows that end with "-resumen.pdf"
data_resumen = data[ data[ 'Name' ].str.endswith( '-resumen.pdf' ) ]
data_resumen[ 'Name' ] = data_resumen['Name'].str.replace( '-resumen.pdf', '' )
data_resumen = data_resumen.rename( columns = { 'URL': 'Resumen' } )

C:\Users\dell\.conda\envs\myenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\dell\.conda\envs\myenv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\dell\.conda\envs\myenv\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  
C:\Users\dell\.conda\envs\myenv\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a

In [71]:
final_data = data_informe.merge( data_resumen, how = 'inner', on = 'Name' )
final_data.to_excel( 'reportes_links.xlsx', index = False )